# Context Managers in Python

Many resources that your programs interact with have rules about how they are to be properly "opened" and "closed", wehre those words loosely represent the set-up and tear-down phases of the interaction.

For example, when you interact with a database, there are specific steps you should perform to *connect*, and specific steps you should perform to *disconnect*. Regardless of what you are doing in between, you need to always disconnect if you have successfully connected.

Because the pattern is well-known and exists in so many ways, the Python language offers a mechanism for ensuring both parts are executed correctly: **context managers**. A context manager is an object (or function decorator) that allows you to wrap your "in between" code in a code block that only starts if setup completes, and that guarantees tear-down regardless of how the block exits (normal return, exception, whatever).

## Example Use

The following example shows the use of a context manager that we are almost all very familiar with:

```python
filename = 'data.csv'

with open(filename, 'r') as fin:
    data = fin.readlines()
```

That `with` statement is really telling Python that a context manager is about to create a context. The `open()` function is actually a context manager that returns a file handle. That handle is "open" inside the indented code block, and becomes "closed" when the code block exits.

The rough steps that this context manager performs for you are:

1. call the low-level OS function to get a handle to the file
1. put a wrapper around that handle that gives you access to functions on the file, like `read()`, `write()`, and `seek()`
1. return the handle to the calling code
1. ... do the work in the code block until either a value is returned or an exception propagates up ...
1. call the low-level OS function to relenquish the handle to the file back to the available group

## Writing a Context Manager

Our first context manager will be simple: it will print a line of text before the inner block is executed and print another line of text when the inner block completes.

You can write context managers two ways, as a class or as a generator function.

### Class-Based Context Manager

The class approach is easier to understand, so we'll start there:

In [1]:
class WrappedCode:
    
    def __init__(self, name):
        self.name = name
    
    def __enter__(self):
        print(f'class-based context manager entered: {self.name}')
        
    def __exit__(self, type, value, traceback):
        print(f'class-based context manager exited: {self.name} ({type}, {value}, {traceback})')

Now we can instantiate this context manager and see it in action:

In [2]:
with WrappedCode('normal'):
    print('inside the inner code block')

class-based context manager entered: normal
inside the inner code block
class-based context manager exited: normal (None, None, None)


This still works if our inner code throws an exception:

In [3]:
try:
    with WrappedCode('exception'):
        print('start inner block')
        raise Exception('oh no!')
except:
    print('exception handled outside of context manager')

class-based context manager entered: exception
start inner block
class-based context manager exited: exception (<class 'Exception'>, oh no!, <traceback object at 0x10a5832d0>)
exception handled outside of context manager


### Function-Based Context Manager

You can also implement the same context manager by using the `@contextmanager` decorator from `contextlib`. In this case, the decorator decorates a function that is responsible for first running the pre-context code, yielding control to the inner block, and then running the post-context block:

In [4]:
from contextlib import contextmanager

@contextmanager
def wrapped_code(name):
    print(f'function-based context manager entered: {name}')
    try:
        yield
    finally:
        print(f'function-based context manager exited: {name}')

This can be used in the same way as the class-based approach:

In [5]:
with wrapped_code('normal'):
    print('inside the inner code block')

function-based context manager entered: normal
inside the inner code block
function-based context manager exited: normal


In [6]:
try:
    with wrapped_code('exception'):
        print('start inner block')
        raise Exception('oh no!')
except:
    print('exception handled outside of context manager')

function-based context manager entered: exception
start inner block
function-based context manager exited: exception
exception handled outside of context manager


### Returning a Context

Sometimes the context manager itself is part of what the inner block requires. For instance, the `open(*args, **kwargs)` context manager returns an appropriate reader object for the settings specifying text or binary mode. Another example would be a context manager for a database connection returning the connection object for your inner code to interact with.

A context manager can return a value from the `__enter__()` method (for class-based) or `yield` a value (for function-based).

Here's a context manager that adds extra methods that can be called:

In [7]:
class Printer:
    
    def __enter__(self):
        print('context manager entered')
        return self
        
    def __exit__(self, type, value, traceback):
        print('context manager exited')
    
    def print(self, message):
        print('message:', message)

with Printer() as printer:
    printer.print('this is cool!')

context manager entered
message: this is cool!
context manager exited


A similar context manager using a decorated function might look like this:

In [8]:
@contextmanager
def printer():
    def inner_print(message):
        print('message:', message)
    print('context manager entered')
    try:
        yield inner_print
    finally:
        print('context manager exited')
        
with printer() as print_message:
    print_message('this is cool!')

context manager entered
message: this is cool!
context manager exited


## Considerations

A context manager is a very convenient mechanism for ensuring you properly release resources when you're done with them. While exception handlers *can* do the right thing, it becomes the responsibility of the *user* of the package to know what cleanup is required. By using a context manager, you as the package manager have the necessary knowledge to clean up all the resources used in the context.

Many context managers don't return anything at all, and this is completely reasonable for many use cases. But the majority of context managers return some kind of context object. For class-based, they sometimes return `self`, but a complex system might return an object that is not the context manager itself but is managed by it. Function-based context managers can also return any object, so using a complex object to describe everything that can be done within the context is not uncommon.

With that in mind, note that it's a best practice to have your code objects each be responsible for a single thing. With this in mind, it's not unusual to have several context managers called together. For this, they can be nested:

In [9]:
with wrapped_code('wrapper') as wrapper:
    with printer() as print_message:
        print_message('nested context are pretty great...')

function-based context manager entered: wrapper
context manager entered
message: nested context are pretty great...
context manager exited
function-based context manager exited: wrapper


Alternately, you can call them together on a single line (as long as none of them are used as the input to any other)

In [10]:
with wrapped_code('wrapper') as wrapper, printer() as print_message:
    print_message('but flat contexts are even cleaner!')

function-based context manager entered: wrapper
context manager entered
message: but flat contexts are even cleaner!
context manager exited
function-based context manager exited: wrapper


When you flatten multiple contexts into a single line, they are applied in the order specified and they unwind in the opposite order they were created, just as they would if they were nested.

## Summary

Context managers give you a simple way to wrap a block of code with the correct setup and tear-down structure to ensure you never forget a critical clean-up step. They work properly with both normal execution and raised exceptions. And they are very simple to write and use.

## References

* [`contextlib` — Utilities for with-statement contexts](https://docs.python.org/3/library/contextlib.html)
* [PEP 343 -- The "with" statement](https://www.python.org/dev/peps/pep-0343/)